### Example: How many files are covered by each type of software license?
GitHub is the most popular place to collaborate on software projects. A GitHub repository (or repo) is a collection of files associated with a specific project.
Most repos on GitHub are shared under a specific legal license, which determines the legal restrictions on how they are used. 

Two tables in the database. 
- The first table is the licenses table, which provides the name of each GitHub repo (in the repo_name column) and its corresponding license. 
- The second table is the sample_files table, which provides, among other information, the GitHub repo that each file belongs to (in the repo_name column). 

In [1]:
from google.cloud import bigquery

# Create a "Client" object
client = bigquery.Client()

# Construct a reference to the "github_repos" dataset
dataset_ref = client.dataset("github_repos", project="bigquery-public-data")

# API request - fetch the dataset
dataset = client.get_dataset(dataset_ref)

# Construct a reference to the "licenses" table
licenses_ref = dataset_ref.table("licenses")

# API request - fetch the table
licenses_table = client.get_table(licenses_ref)

# Preview the first five lines of the "licenses" table
client.list_rows(licenses_table, max_results=5).to_dataframe()

Using Kaggle's public dataset BigQuery integration.


,repo_name,license
0,kate-mcgaughey/reddit-comments,artistic-2.0
1,titaniumtails/bellatrix,artistic-2.0
2,gdude2002/Quiz,artistic-2.0
3,fowlers48/StepOne,artistic-2.0
4,ludovicc/datasciencecoursera,artistic-2.0


In [2]:
# Construct a reference to the "sample_files" table
files_ref = dataset_ref.table("sample_files")

# API request - fetch the table
files_table = client.get_table(files_ref)

# Preview the first five lines of the "sample_files" table
client.list_rows(files_table, max_results=10).to_dataframe()

,repo_name,ref,path,mode,id,symlink_target
0,git/git,refs/heads/master,RelNotes,40960,62615ffa4e97803da96aefbc798ab50f949a8db7,Documentation/RelNotes/2.10.0.txt
1,np/ling,refs/heads/master,tests/success/plug_compose.t/plug_compose.ll,40960,0c1605e4b447158085656487dc477f7670c4bac1,../../../fixtures/all/plug_compose.ll
2,np/ling,refs/heads/master,fixtures/strict-par-success/parallel_assoc_lef...,40960,b59bff84ec03d12fabd3b51a27ed7e39a180097e,../all/parallel_assoc_left.ll
3,np/ling,refs/heads/master,fixtures/sequence/parallel_assoc_2tensor2_left.ll,40960,f29523e3fb65702d99478e429eac6f801f32152b,../all/parallel_assoc_2tensor2_left.ll
4,np/ling,refs/heads/master,fixtures/success/my_dual.ll,40960,38a3af095088f90dfc956cb990e893909c3ab286,../all/my_dual.ll
5,np/ling,refs/heads/master,tests/success/literals.t/literals.ll,40960,e933ac4bb885bf2ce6dddf01853a61310a684dd3,../../../fixtures/all/literals.ll
6,np/ling,refs/heads/master,fixtures/sequence/feed_recv.ll,40960,3f0bd39ae146acade476d76a3f0ebd391974c570,../all/feed_recv.ll
7,np/ling,refs/heads/master,tests/success/lettype.t/lettype.ll,40960,065788982da09b08b8defc3e3b0f9764fae2a7b9,../../../fixtures/all/lettype.ll
8,np/ling,refs/heads/master,fixtures/strict-par-success/literals.ll,40960,3cf9f78c9276d38d6192149d2d30d4b4b0dba119,../all/literals.ll
9,np/ling,refs/heads/master,fixtures/success/parallel_assoc_tensor3_left.ll,40960,0c491fa21a5fdabfd08392e7f45e70fae33cc9f9,../all/parallel_assoc_tensor3_left.ll


In [3]:
# Query to determine the number of files per license, sorted by number of files
query = """
        SELECT L.license, COUNT(1) AS number_of_files
        FROM `bigquery-public-data.github_repos.sample_files` AS sf
        INNER JOIN `bigquery-public-data.github_repos.licenses` AS L 
            ON sf.repo_name = L.repo_name
        GROUP BY L.license
        ORDER BY number_of_files DESC
        """
# count the number of rows with COUNT(1)

# Set up the query (cancel the query if it would use too much of your quota, with the limit set to 10 GB)
safe_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10)
query_job = client.query(query, job_config=safe_config)

# API request - run the query, and convert the results to a pandas DataFrame
file_count_by_license = query_job.to_dataframe()

In [4]:
file_count_by_license.head(10)

,license,number_of_files
0,mit,20624106
1,gpl-2.0,17120293
2,apache-2.0,7232495
3,gpl-3.0,4961351
4,bsd-3-clause,2944904
5,agpl-3.0,1309571
6,lgpl-2.1,801882
7,bsd-2-clause,700697
8,lgpl-3.0,567925
9,mpl-2.0,468372
